In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('balanced_df.csv')
df.head(-10)

,reviewer_id,store_location,latitude,longitude,date,month,year,title,review,review-label
0,151782.0,US,37.090240,-95.712891,2021,6,2008 00:00:00,I like the tee shirt I ordered butï¿½ï¿½ï¿½ï¿½...,like tee shirt order truth qualiti shirt self ...,1
1,27155.0,US,37.090240,-95.712891,2023,1,2001 00:00:00,Poor quality and smelled bad,order two differ item differ vendor shirt chea...,1
2,72707.0,US,37.090240,-95.712891,2022,8,2007 00:00:00,I did not get my complete order,get complet order im miss mask,1
3,2766.0,US,37.090240,-95.712891,2023,6,2010 00:00:00,I never received the shirt I orderedï¿½ï¿½ï¿½,never receiv shirt order get touch anyon,1
4,240808.0,US,37.090240,-95.712891,2019,9,2028 00:00:00,"Too small, poor quality",normal xl order xl could put long sleev shirt ...,1
...,...,...,...,...,...,...,...,...,...,...
53630,95978.0,CA,56.130366,-106.346771,2022,5,2001 00:00:00,Love it,bought friend one age ago lost arriv within we...,5
53631,188654.0,US,37.090240,-95.712891,2020,9,2023 00:00:00,Fridge magnet and lapel pin,item arriv good item undamag exactli pictur pl...,5
53632,278034.0,CA,56.130366,-106.346771,2018,4,2027 00:00:00,Great quality,great qualiti love shirt even splurg premium o...,5
53633,12182.0,US,37.090240,-95.712891,2023,4,2016 00:00:00,Punctuality was phenomenal,punctual phenomen got everyth expect,5


In [ ]:
X = list(df['review'])
Y = df['review-label']

In [ ]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

In [ ]:
x_train, x_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
x_train, x_val, Y_train, Y_val = train_test_split(x_train, Y_train, test_size=0.25, random_state=42)


In [ ]:
x_train = [str(x) for x in x_train if isinstance(x, (str, int, float))]
x_val = [str(x) for x in x_val if isinstance(x, (str, int, float))]
x_test = [str(x) for x in x_test if isinstance(x, (str, int, float))]
#

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize_data(texts, tokenizer, max_length=64):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

In [ ]:
train_encodings = tokenize_data(x_train, tokenizer)
val_encodings = tokenize_data(x_val, tokenizer)

In [ ]:
train_encodings['input_ids'][602].shape


TensorShape([64])

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_encodings['input_ids'], np.array(Y_train),
    validation_data=(val_encodings['input_ids'], np.array(Y_val)),
    epochs=10,
    batch_size=32
)

Epoch 1/10
1006/1006 [==============================] - 538s 487ms/step - loss: 1.1830 - accuracy: 0.4772 - val_loss: 1.0603 - val_accuracy: 0.5382
Epoch 2/10
1006/1006 [==============================] - 489s 486ms/step - loss: 0.9922 - accuracy: 0.5741 - val_loss: 1.0382 - val_accuracy: 0.5432
Epoch 3/10
1006/1006 [==============================] - 489s 486ms/step - loss: 0.8812 - accuracy: 0.6229 - val_loss: 1.0577 - val_accuracy: 0.5571
Epoch 4/10
1006/1006 [==============================] - 489s 486ms/step - loss: 0.7468 - accuracy: 0.6899 - val_loss: 1.1513 - val_accuracy: 0.5474
Epoch 5/10
1006/1006 [==============================] - 488s 486ms/step - loss: 0.6030 - accuracy: 0.7604 - val_loss: 1.2727 - val_accuracy: 0.5394
Epoch 6/10
1006/1006 [==============================] - 488s 486ms/step - loss: 0.4775 - accuracy: 0.8145 - val_loss: 1.4296 - val_accuracy: 0.5454
Epoch 7/10
1006/1006 [==============================] - 489s 486ms/step - loss: 0.3756 - accuracy: 0.8579 - val_

In [ ]:
# Evaluate the model on validation set
results = model.evaluate(val_encodings['input_ids'], np.array(Y_val))
print(f"Validation Accuracy: {results[1]:.4f}")

336/336 [==============================] - 48s 143ms/step - loss: 2.0588 - accuracy: 0.5148
Validation Accuracy: 0.5148


In [ ]:
test_prediction = model.predict(val_encodings['input_ids'])

336/336 [==============================] - 49s 138ms/step
